<a href="https://colab.research.google.com/github/hasiburrahman1/NLP/blob/master/MachineTranslationEn_Bn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Translation English to Bangla with RNN

Machine Translation using Recurrent Neural Network
> License https://www.apache.org/licenses/LICENSE-2.0

## Environment Setup
- Install Anaconda `Jupyter` Notebook.
- Install Python3.
- With Anaconda Prompt install `Tensorflow Keras` NLP Tools. For Faster Processing need GPU processing. 
    - Keras
```batch
   conda install -c conda-forge tensorflow
```

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import tensorflow as tf
import collections
import unicodedata
import re
import numpy as np
import os
import io
import time

In [ ]:
# check up GPU/CPU processing power

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6515303086157523200
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 18017270970326380120
physical_device_desc: "device: XLA_CPU device"
]


In [ ]:
# dataset path
path_to_file = "data/datasetCleaned.txt"

In [ ]:
def preprocess_sentence(w):
#     remove punctuations from lines
    w = re.sub(r"([?.!,¿।])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [ ]:
def vocabularyCounter(dataFileEn, dataFileBn):
    # setup Vocabulary 
    english_words = collections.Counter([word for sentence in dataFileEn for word in sentence.split()])
    bangla_words = collections.Counter([word for sentence in dataFileBn for word in sentence.split()])
    
    print('Total English words: {}'.format(len([word for sentence in dataFileEn for word in sentence.split()])))
    print('Unique English words: {}'.format(len(english_words)))
    print('Most Common Words: "' + '" "'.join(list(zip(*english_words.most_common(10)))[0]) + '"\n')

    print('Total Bangla words: {}'.format(len([word for sentence in dataFileBn for word in sentence.split()])))
    print('Unique Bangla words: {}'.format(len(bangla_words)))
    print('Most Common Words: "' + '" "'.join(list(zip(*bangla_words.most_common(10)))[0]) + '"')

In [ ]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, BANGLA]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    allEnLines = []
    allBnLines = []
    for l in lines[:num_examples]:
        w = l.split('\t')
        allEnLines.append(w[0])
        allBnLines.append(w[1])
    vocabularyCounter(allEnLines, allBnLines) 
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    
#     rearrange dataset src and target
    for indd in range(len(word_pairs)):
        temp = word_pairs[indd][0]
        word_pairs[indd][0] = word_pairs[indd][1]
        word_pairs[indd][1] = temp
    return zip(*word_pairs)

In [ ]:
# preprocessing function declear and vocabulary checking
bn, en = create_dataset(path_to_file, None)
print("\n\n", en[20])
print(bn[20])

Total English words: 19606
Unique English words: 2839
Most Common Words: "I" "Tom" "to" "you" "is" "the" "a" "I'm" "your" "He"

Total Bangla words: 19000
Unique Bangla words: 3527
Most Common Words: "আমি" "টম" "আমার" "কি" "না।" "আপনি" "এটা" "তুমি" "কথা" "আপনার"


 <start> Got it ! <end>
<start> বুঝে গেছি ! <end>


## Tokenize

In [ ]:
#function declearation for 
#     1. length of sentence

def max_length(tensor):
    return max(len(t) for t in tensor)

#     2. tokenizer
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer
#     3. load dataset
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang  = create_dataset(path, num_examples)
    
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

## Training Dataset

In [ ]:
# Try experimenting with the size of that dataset
num_examples = 1000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print("Size:- Train_X-", len(input_tensor_train), "Train_Y-", len(target_tensor_train), "Test_X-", len(input_tensor_val), "Test_Y-", len(target_tensor_val))

Total English words: 2425
Unique English words: 555
Most Common Words: "I" "Tom" "Tom." "I'm" "is" "He" "Who" "You" "me." "it."

Total Bangla words: 2645
Unique Bangla words: 770
Most Common Words: "আমি" "টম" "আমার" "টমকে" "আমরা" "আমাকে" "না।" "এটা" "কি" "করুন।"
Size:- Train_X- 800 Train_Y- 800 Test_X- 200 Test_Y- 200


In [ ]:
# showing example of tokenization
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))
    
    
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
4 ----> i
209 ----> love
7 ----> you
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
4 ----> আমি
78 ----> তোমাকে
483 ----> ভালবাসি
3 ----> ।
2 ----> <end>


In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
print("Input-output shape batch", example_input_batch.shape, example_target_batch.shape)

Input-output shape batch (64, 7) (64, 9)


In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units, activation='linear',return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 7, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
# Attention and context_vector
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    hidden_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.sigmoid(self.W1(values) + self.W2(hidden_with_time_axis)))
    
    attention_weights = tf.nn.sigmoid(score)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 7, 1)


In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units, activation='tanh',return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)

    x = self.embedding(x)

    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)
    
    output = tf.reshape(output, (-1, output.shape[2]))
    x = self.fc(output)

    return x, state, attention_weights


decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)), sample_hidden, sample_output)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 673)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoint saving object 

In [ ]:
# Checkpoint for saving the object
checkpoint_dir = './training_checkpoints_saved'
checkpoint_prefix = os.path.join(checkpoint_dir, "checkpointSaved")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
# start learn processing
EPOCHS = 100
lossMatrix = []
for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
    lossMatrix.append(total_loss / steps_per_epoch)
    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.2451
Epoch 2 Batch 0 Loss 2.1686
Epoch 2 Loss 2.1079
Time taken for 1 epoch 20.16243553161621 sec

Epoch 3 Batch 0 Loss 1.9826
Epoch 4 Batch 0 Loss 1.8511
Epoch 4 Loss 1.7214
Time taken for 1 epoch 20.92720651626587 sec

Epoch 5 Batch 0 Loss 1.5548
Epoch 6 Batch 0 Loss 1.3135
Epoch 6 Loss 1.3241
Time taken for 1 epoch 17.756619691848755 sec

Epoch 7 Batch 0 Loss 1.2623
Epoch 8 Batch 0 Loss 1.0449
Epoch 8 Loss 1.0087
Time taken for 1 epoch 19.9654221534729 sec

Epoch 9 Batch 0 Loss 0.8683
Epoch 10 Batch 0 Loss 0.7822
Epoch 10 Loss 0.7303
Time taken for 1 epoch 17.951283931732178 sec

Epoch 11 Batch 0 Loss 0.6162
Epoch 12 Batch 0 Loss 0.5159
Epoch 12 Loss 0.5253
Time taken for 1 epoch 17.152433156967163 sec

Epoch 13 Batch 0 Loss 0.4394
Epoch 14 Batch 0 Loss 0.3263
Epoch 14 Loss 0.3647
Time taken for 1 epoch 19.278913497924805 sec

Epoch 15 Batch 0 Loss 0.2828
Epoch 16 Batch 0 Loss 0.2656
Epoch 16 Loss 0.2722
Time taken for 1 epoch 15.174743175506592 sec

Epoch 17 

In [ ]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
#     plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
# load checklist object from saving
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate("let me leave !")

Input: <start> let me leave ! <end>
Predicted translation: আমাকে যেতে দাও ! <end> 
